In [1]:
import numpy as np
import os

import pickle
import random
import subprocess
from tqdm.notebook import tqdm_notebook

In [3]:
output_folder = '4_11_actualq4_merged'
data_folders = ['4_11_actualq4']

In [4]:
directories = ['../data/' + data_folder for data_folder in data_folders]

filenames = []
for directory in directories:
    filenames.extend([directory + '/' + file for file in os.listdir(directory) if 'all.npz' in file])

In [5]:
filenames

['../data/4_11_actualq4/all.npz']

In [6]:
collected_history_matrices = []
collected_move_vectors = []
collected_hand_vectors = []
collected_y = []

for filename in tqdm_notebook(filenames):
    with np.load(filename) as npzfile:
        collected_history_matrices.append(npzfile['history_matrix'])
        collected_move_vectors.append(npzfile['move_vectors'])
        collected_hand_vectors.append(npzfile['hand_vectors'])
        collected_y.append(npzfile['y'])

In [7]:
# if stacking is needed
if len(collected_y) > 1:
    all_y = np.hstack(collected_y)
    all_move_vectors = np.vstack(collected_move_vectors)
    all_hand_vectors = np.vstack(collected_hand_vectors)
    all_history_matrices = np.vstack(collected_history_matrices)
else:
    all_y = collected_y[0]
    all_move_vectors = collected_move_vectors[0]
    all_hand_vectors = collected_hand_vectors[0]
    all_history_matrices = collected_history_matrices[0]

In [8]:
from sklearn.model_selection import train_test_split
train_hm, test_hm, train_mv, test_mv, train_hv, test_hv, train_y, test_y = \
    train_test_split(all_history_matrices, all_move_vectors, all_hand_vectors, all_y, test_size=0.1, shuffle=True)

In [9]:
!rm -r ../data/{output_folder}/
!mkdir ../data/{output_folder}/

rm: cannot remove '../data/4_11_actualq4_merged/': No such file or directory


In [10]:
np.savez('../data/' + output_folder + '/train.npz', 
         history_matrix=train_hm,
         move_vectors=train_mv,
         hand_vectors=train_hv,
         y=train_y)

np.savez('../data/' + output_folder + '/test.npz',
         history_matrix=test_hm,
         move_vectors=test_mv,
         hand_vectors=test_hv,
         y=test_y)

In [11]:
!gsutil -m cp -r ../data/{output_folder}/ gs://landlord_ai/

Copying file://../data/4_11_actualq4_merged/test.npz [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../data/4_11_actualq4_merged/train.npz [Content-Type=application/octet-stream]...
/ [2/2 files][ 14.6 GiB/ 14.6 GiB] 100% Done  96.9 MiB/s ETA 00:00:00           
Operation completed over 2 objects/14.6 GiB.      